In [1]:
# Importing dependencies
import pandas as pd
from sqlalchemy import create_engine
from geopy.exc import GeocoderTimedOut
from geopy.geocoders import Nominatim
import numpy as np

In [3]:
# Creating dataframe from csv file
csv_file = "Resources/wine_data.csv"
wine_df=pd.read_csv(csv_file, encoding="utf-8")
#wine_df.head()

In [4]:
# Resetting index
wine_df.reset_index()
# Removed unnecessary columns (Description and Designation) and created a copy
new_wine_data = wine_df[["country", "points", "price", "province", "region_1", "region_2", "taster_name",\
                        "taster_twitter_handle", "title", "variety", "winery"]].copy()
#new_wine_data.head()

In [5]:
# dropping the duplicate records and keeping the first record
new_wine_data=new_wine_data.drop_duplicates(keep='first')
new_wine_data[new_wine_data.duplicated(keep=False)]

,country,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery


In [6]:
# Dropped null records in country column
new_wine_data=new_wine_data.loc[(new_wine_data["country"].isna()==False),:]
new_wine_data.head()

,country,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [8]:
#Creating province_df with province column; lat and long values will be added to the dataframe
province_df=new_wine_data[["province"]].drop_duplicates().reset_index().drop(columns=["index"])
province_df.columns=["province"]
province_df.head()

,province
0,Sicily & Sardinia
1,Douro
2,Oregon
3,Michigan
4,Northern Spain


In [9]:
# Calculating latitude and longtitude for provinces in province dataframe  and assigning to latitude and longitude list

longitude = [] 
latitude = [] 
   
# function to find the coordinate of a given province
def findGeocode(province): 
       
    geolocator = Nominatim(user_agent="wine_review") 

    return geolocator.geocode(province,timeout=None) 
    

    # finding lat and long values for each province by calling findGeocode() function 
    #and passing province from province_df dataframe
for i in (province_df["province"]): 
      
    if findGeocode(i) != None: 
           
        loc = findGeocode(i) 
          
        # coordinates returned from  
        # function is stored into 
        # two separate list 
        latitude.append(loc.latitude) 
        longitude.append(loc.longitude) 
       
    # if coordinate for a city not 
    # found, insert "NaN" indicating  
    # missing value  
    else: 
        latitude.append(np.nan) 
        longitude.append(np.nan) 

In [10]:
#Assigning latitudes and longitudes created for province to respective columns in province dataframe
province_df["latitude"]=latitude
province_df["longitude"]=latitude

In [13]:
#Creating csv file containing province and its latitude and longitude position
province_df.to_csv("Resources\province_lat_long.csv", index = False)